# Analytics Export

Export database analytics to JSON for admin dashboard.

In [1]:
# 📚 Imports necessários
import mysql.connector
import json
from datetime import datetime, timedelta
import pandas as pd
import os

print("✅ Bibliotecas importadas!")
print(f"📅 Executado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Database connection
try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='db_noted'
    )
    cursor = conn.cursor()
    print("✅ Conexão à base de dados estabelecida")
except Exception as e:
    print(f"❌ Erro na conexão: {e}")
    exit()

# Helper function to check connection and reconnect if needed
def ensure_connection():
    global conn, cursor
    try:
        # Check if connection is alive
        conn.ping(reconnect=False, attempts=1, delay=0)
    except:
        print("⚠️ Conexão perdida. A reconectar...")
        try:
            # Try to reconnect
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='db_noted'
            )
            cursor = conn.cursor()
            print("✅ Reconexão bem-sucedida")
        except Exception as e:
            print(f"❌ Falha ao reconectar: {e}")
            exit()

# Helper function for safe query execution
def safe_query(query, params=None):
    try:
        ensure_connection()  # Ensure connection is active
        cursor.execute(query, params)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(f"❌ Erro na query: {e}")
        print(f"Query: {query}")
        return []

# Calculate date ranges
today = datetime.now().date()
month_start = today.replace(day=1)
last_month = (month_start - timedelta(days=1)).replace(day=1)
year_start = today.replace(month=1, day=1)

print(f"📅 Período de análise:")
print(f"   Hoje: {today}")
print(f"   Início do mês: {month_start}")
print(f"   Início do ano: {year_start}")

# ANALYTICS DATA STRUCTURE
analytics_data = {
    "last_updated": datetime.now().isoformat(),
    "dashboard": {
        "total_sales": 0,
        "monthly_sales": 0,
        "total_orders": 0,
        "monthly_orders": 0,
        "total_customers": 0,
        "active_products": 0,
        "revenue_growth": 0,
        "recent_orders": []
    },
    "products": {
        "total_products": 0,
        "total_categories": 0,
        "out_of_stock_count": 0,
        "low_stock_count": 0,
        "products_by_category": [],
        "top_selling_products": [],
        "low_stock_products": [],
        "out_of_stock_products": [],
        "recent_products": []
    },
    "orders": {
        "total_orders": 0,
        "monthly_orders": 0,
        "paid_orders": 0,
        "unpaid_orders": 0,
        "orders_by_month": [],
        "orders_by_payment_method": [],
        "recent_orders": []
    },
    "users": {
        "total_users": 0,
        "total_customers": 0,
        "total_admins": 0,
        "recent_registrations": 0,
        "users_with_orders": 0,
        "top_customers": []
    },
    "analytics": {
        "sales_by_month": [],
        "orders_by_day": [],
        "top_products": [],
        "customer_analytics": [],
        "payment_methods": [],
        "monthly_revenue": []
    }
}

print("\n🔍 A recolher dados analytics...")


✅ Bibliotecas importadas!
📅 Executado em: 2025-07-06 20:34:05
✅ Conexão à base de dados estabelecida
📅 Período de análise:
   Hoje: 2025-07-06
   Início do mês: 2025-07-01
   Início do ano: 2025-01-01

🔍 A recolher dados analytics...


# Setup

Import libraries, connect to database, define data structure.

# Overview

Collects analytics from database → exports to `/static/data/analytics.json`

Data: sales, orders, products, users, analytics

In [2]:

# =============================================================================
# DASHBOARD ANALYTICS
# =============================================================================

# Total sales (all time)
query = "SELECT COALESCE(SUM(total), 0) as total_sales FROM orders"
result = safe_query(query)
analytics_data["dashboard"]["total_sales"] = float(result[0][0]) if result else 0

# Monthly sales (current month)
query = "SELECT COALESCE(SUM(total), 0) as monthly_sales FROM orders WHERE DATE(order_date) >= %s"
result = safe_query(query, (month_start,))
analytics_data["dashboard"]["monthly_sales"] = float(result[0][0]) if result else 0

# Total orders
query = "SELECT COUNT(*) as total_orders FROM orders"
result = safe_query(query)
analytics_data["dashboard"]["total_orders"] = result[0][0] if result else 0

# Monthly orders
query = "SELECT COUNT(*) as monthly_orders FROM orders WHERE DATE(order_date) >= %s"
result = safe_query(query, (month_start,))
analytics_data["dashboard"]["monthly_orders"] = result[0][0] if result else 0

# Total customers (users with role=2)
query = "SELECT COUNT(*) as total_customers FROM users WHERE role = 2"
result = safe_query(query)
analytics_data["dashboard"]["total_customers"] = result[0][0] if result else 0

# Active products
query = "SELECT COUNT(*) as active_products FROM products"
result = safe_query(query)
analytics_data["dashboard"]["active_products"] = result[0][0] if result else 0

# Revenue growth (compare current month to last month)
query = """
SELECT 
    COALESCE(SUM(CASE WHEN DATE(order_date) >= %s THEN total ELSE 0 END), 0) as current_month,
    COALESCE(SUM(CASE WHEN DATE(order_date) >= %s AND DATE(order_date) < %s THEN total ELSE 0 END), 0) as last_month
FROM orders
"""
result = safe_query(query, (month_start, last_month, month_start))
if result and result[0][1] > 0:
    current_month_sales = float(result[0][0])
    last_month_sales = float(result[0][1])
    analytics_data["dashboard"]["revenue_growth"] = round(((current_month_sales - last_month_sales) / last_month_sales) * 100, 2)

# Recent orders (last 10)
query = """
SELECT o.id, u.name, o.total, o.order_date, p.paid
FROM orders o
LEFT JOIN users u ON o.user_id = u.id
LEFT JOIN payment_info p ON o.id = p.order_id
ORDER BY o.order_date DESC
LIMIT 10
"""
result = safe_query(query)
for row in result:
    analytics_data["dashboard"]["recent_orders"].append({
        "id": row[0],
        "customer": row[1] or "Cliente Anônimo",
        "total": float(row[2]) if row[2] else 0,
        "date": row[3].isoformat() if row[3] else None,
        "paid": bool(row[4]) if row[4] is not None else False
    })

print(f"✅ Dashboard: {analytics_data['dashboard']['total_sales']}€ vendas totais, {analytics_data['dashboard']['total_orders']} encomendas")


✅ Dashboard: 3897.92€ vendas totais, 22 encomendas


# Dashboard Analytics

Main dashboard metrics: sales, orders, customers, growth.

**Metrics:**
- Total sales (all time)
- Monthly sales
- Order counts (total and monthly)
- Customer count
- Product count
- Revenue growth (month-over-month)
- Recent orders (last 10)

**Tables used:** orders, users, products, payment_info

In [3]:
# =============================================================================
# PRODUCTS ANALYTICS
# =============================================================================

# safe_query function returns an empty list if the query fails for some reason (e.g., database connection issues).
# Total products
query = "SELECT COUNT(*) FROM products"
result = safe_query(query) 
# normal if statement, if result is empty, set total_products to 0
analytics_data["products"]["total_products"] = result[0][0] if result else 0
# no need to use a try-except block here, as safe_query already handles errors

# Total categories
query = "SELECT COUNT(*) FROM categories"
result = safe_query(query)
analytics_data["products"]["total_categories"] = result[0][0] if result else 0

# Products by category
query = """
SELECT c.description, COUNT(p.id) as product_count
FROM categories c
LEFT JOIN products p ON c.id = p.category_id
GROUP BY c.id, c.description
ORDER BY product_count DESC
"""
result = safe_query(query)
for row in result:
    analytics_data["products"]["products_by_category"].append({
        "category": row[0],
        "count": row[1]
    })

# Top selling products (based on order items)
query = """
SELECT p.name, SUM(oi.quantity) as total_sold, SUM(oi.quantity * oi.unit_price) as total_revenue
FROM products p
JOIN order_items oi ON p.id = oi.product_id
GROUP BY p.id, p.name
ORDER BY total_sold DESC
LIMIT 10
"""
result = safe_query(query)
for row in result:
    analytics_data["products"]["top_selling_products"].append({
        "name": row[0],
        "quantity_sold": row[1],
        "revenue": float(row[2])
    })

# Stock Analytics - Out of Stock and Low Stock Products
# Out of stock products (quantity = 0)
query = """
SELECT COUNT(*) as out_of_stock_count
FROM product_stock 
WHERE quantity = 0
"""
result = safe_query(query)
analytics_data["products"]["out_of_stock_count"] = result[0][0] if result else 0

# Low stock products (quantity <= 10 but > 0)
query = """
SELECT COUNT(*) as low_stock_count
FROM product_stock 
WHERE quantity > 0 AND quantity <= 10
"""
result = safe_query(query)
analytics_data["products"]["low_stock_count"] = result[0][0] if result else 0

# Low stock products details (for admin alerts)
query = """
SELECT p.name, ps.quantity, c.description as category
FROM products p
JOIN product_stock ps ON p.id = ps.product_id
LEFT JOIN categories c ON p.category_id = c.id
WHERE ps.quantity > 0 AND ps.quantity <= 10
ORDER BY ps.quantity ASC, p.name
LIMIT 20
"""
result = safe_query(query)
for row in result:
    analytics_data["products"]["low_stock_products"].append({
        "name": row[0],
        "quantity": row[1],
        "category": row[2] or "Sem categoria"
    })

# Out of stock products details
query = """
SELECT p.name, c.description as category
FROM products p
JOIN product_stock ps ON p.id = ps.product_id
LEFT JOIN categories c ON p.category_id = c.id
WHERE ps.quantity = 0
ORDER BY p.name
LIMIT 20
"""
result = safe_query(query)
analytics_data["products"]["out_of_stock_products"] = []
for row in result:
    analytics_data["products"]["out_of_stock_products"].append({
        "name": row[0],
        "category": row[1] or "Sem categoria"
    })

# Recent products (last 10)
query = """
SELECT p.name, p.price, p.created_at, c.description
FROM products p
LEFT JOIN categories c ON p.category_id = c.id
ORDER BY p.created_at DESC
LIMIT 10
"""
result = safe_query(query)
for row in result:
    analytics_data["products"]["recent_products"].append({
        "name": row[0],
        "price": float(row[1]),
        "created_at": row[2].isoformat() if row[2] else None,
        "category": row[3] or "Sem categoria"
    })

print(f"✅ Produtos: {analytics_data['products']['total_products']} produtos, {analytics_data['products']['total_categories']} categorias")
print(f"📦 Stock: {analytics_data['products']['out_of_stock_count']} esgotados, {analytics_data['products']['low_stock_count']} stock baixo")


✅ Produtos: 12 produtos, 4 categorias
📦 Stock: 0 esgotados, 0 stock baixo


# Products Analytics

Product counts, categories, top sellers, and stock management.

**Metrics:**
- Total products
- Total categories
- **Stock Analytics:**
  - Out of stock products count (quantity = 0)
  - Low stock products count (quantity ≤ 10)
  - Out of stock products list
  - Low stock products list with quantities
- Products by category
- Top selling products
- Recent products (last 10)

**Tables used:** products, categories, order_items, **product_stock**

In [4]:

# =============================================================================
# ORDERS ANALYTICS
# =============================================================================

# Total orders
analytics_data["orders"]["total_orders"] = analytics_data["dashboard"]["total_orders"]

# Monthly orders
analytics_data["orders"]["monthly_orders"] = analytics_data["dashboard"]["monthly_orders"]

# Paid vs unpaid orders
query = """
SELECT 
    SUM(CASE WHEN p.paid = 1 THEN 1 ELSE 0 END) as paid_orders,
    SUM(CASE WHEN p.paid = 0 OR p.paid IS NULL THEN 1 ELSE 0 END) as unpaid_orders
FROM orders o
LEFT JOIN payment_info p ON o.id = p.order_id
"""
result = safe_query(query)
if result:
    analytics_data["orders"]["paid_orders"] = result[0][0] or 0
    analytics_data["orders"]["unpaid_orders"] = result[0][1] or 0

# Orders by month (last 12 months)
query = """
SELECT 
    DATE_FORMAT(order_date, '%Y-%m') as month,
    COUNT(*) as order_count,
    SUM(total) as total_revenue
FROM orders
WHERE order_date >= DATE_SUB(NOW(), INTERVAL 12 MONTH)
GROUP BY DATE_FORMAT(order_date, '%Y-%m')
ORDER BY month
"""
result = safe_query(query)
for row in result:
    analytics_data["orders"]["orders_by_month"].append({
        "month": row[0],
        "count": row[1],
        "revenue": float(row[2]) if row[2] else 0
    })

# Orders by payment method
query = """
SELECT o.payment_method, COUNT(*) as count
FROM orders o
WHERE o.payment_method IS NOT NULL
GROUP BY o.payment_method
ORDER BY count DESC
"""
result = safe_query(query)
for row in result:
    analytics_data["orders"]["orders_by_payment_method"].append({
        "method": row[0],
        "count": row[1]
    })

# Recent orders (same as dashboard)
analytics_data["orders"]["recent_orders"] = analytics_data["dashboard"]["recent_orders"]

print(f"✅ Encomendas: {analytics_data['orders']['total_orders']} total, {analytics_data['orders']['paid_orders']} pagas")


✅ Encomendas: 22 total, 18 pagas


# Orders Analytics

Order counts, payment status, trends by month.

**Metrics:**
- Order counts (total and monthly)
- Payment status (paid vs unpaid)
- Orders by month (12 months)
- Payment methods
- Recent orders

**Tables used:** orders, payment_info, users

In [5]:

# =============================================================================
# USERS ANALYTICS
# =============================================================================

# Total users
query = "SELECT COUNT(*) FROM users"
result = safe_query(query)
analytics_data["users"]["total_users"] = result[0][0] if result else 0

# Customers vs admins
query = """
SELECT 
    SUM(CASE WHEN role = 2 THEN 1 ELSE 0 END) as customers,
    SUM(CASE WHEN role = 1 THEN 1 ELSE 0 END) as admins
FROM users
"""
result = safe_query(query)
if result:
    analytics_data["users"]["total_customers"] = result[0][0] or 0
    analytics_data["users"]["total_admins"] = result[0][1] or 0

# Recent registrations (last 30 days)
query = "SELECT COUNT(*) FROM users WHERE id > (SELECT MAX(id) - 30 FROM users)"
result = safe_query(query)
analytics_data["users"]["recent_registrations"] = result[0][0] if result else 0

# Users with orders
query = "SELECT COUNT(DISTINCT user_id) FROM orders WHERE user_id IS NOT NULL"
result = safe_query(query)
analytics_data["users"]["users_with_orders"] = result[0][0] if result else 0

# Top customers by total spent
query = """
SELECT u.name, u.email, SUM(o.total) as total_spent, COUNT(o.id) as order_count
FROM users u
JOIN orders o ON u.id = o.user_id
GROUP BY u.id, u.name, u.email
ORDER BY total_spent DESC
LIMIT 10
"""
result = safe_query(query)
for row in result:
    analytics_data["users"]["top_customers"].append({
        "name": row[0],
        "email": row[1],
        "total_spent": float(row[2]),
        "order_count": row[3]
    })

print(f"✅ Utilizadores: {analytics_data['users']['total_users']} total, {analytics_data['users']['total_customers']} clientes")


✅ Utilizadores: 13 total, 11 clientes


# Users Analytics

User counts, roles (1=admin, 2=customer), top customers.

**Metrics:**
- Total users
- User roles (admins vs customers)
- Recent registrations (30 days)
- Users with orders
- Top customers by spending

**Tables used:** users, orders

In [6]:
# =============================================================================
# GENERAL ANALYTICS
# =============================================================================

# Sales by month (last 12 months)
analytics_data["analytics"]["sales_by_month"] = analytics_data["orders"]["orders_by_month"]

# Orders by day (last 30 days)
query = """
SELECT 
    DATE(order_date) as order_day,
    COUNT(*) as order_count,
    SUM(total) as daily_revenue
FROM orders
WHERE order_date >= DATE_SUB(NOW(), INTERVAL 30 DAY)
GROUP BY DATE(order_date)
ORDER BY order_day
"""
result = safe_query(query)
for row in result:
    analytics_data["analytics"]["orders_by_day"].append({
        "date": row[0].isoformat() if row[0] else None,
        "count": row[1],
        "revenue": float(row[2]) if row[2] else 0
    })

# Top products (same as products section)
analytics_data["analytics"]["top_products"] = analytics_data["products"]["top_selling_products"]

# Customer analytics
analytics_data["analytics"]["customer_analytics"] = analytics_data["users"]["top_customers"]

# Payment methods
analytics_data["analytics"]["payment_methods"] = analytics_data["orders"]["orders_by_payment_method"]

# Monthly revenue (same as sales by month)
analytics_data["analytics"]["monthly_revenue"] = [
    {"month": item["month"], "revenue": item["revenue"]} 
    for item in analytics_data["orders"]["orders_by_month"]
]

print(f"✅ Analytics gerais: {len(analytics_data['analytics']['orders_by_day'])} dias de dados")

# Close database connection
cursor.close()
conn.close()

print(f"\n📊 RESUMO DOS DADOS RECOLHIDOS:")
print(f"   💰 Vendas totais: {analytics_data['dashboard']['total_sales']}€")
print(f"   📦 Encomendas totais: {analytics_data['dashboard']['total_orders']}")
print(f"   👥 Clientes totais: {analytics_data['dashboard']['total_customers']}")
print(f"   🛍️ Produtos ativos: {analytics_data['dashboard']['active_products']}")
print(f"   💳 Encomendas pagas: {analytics_data['orders']['paid_orders']}")
print(f"   🎯 Top produtos: {len(analytics_data['products']['top_selling_products'])}")
print(f"   📈 Meses com dados: {len(analytics_data['orders']['orders_by_month'])}")

# Export to JSON with proper decimal handling
output_path = os.path.join("..", "static", "data", "analytics.json")

def decimal_default(obj):
    """JSON serializer for objects not serializable by default json code"""
    if hasattr(obj, '__float__'):
        return float(obj)
    raise TypeError

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(analytics_data, f, indent=2, ensure_ascii=False, default=decimal_default)

print(f"\n✅ Dados exportados para: {output_path}")

# Check file size by reading it back
try:
    with open(output_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    print(f"📄 Tamanho do ficheiro JSON: {len(file_content)} caracteres")
except Exception as e:
    print(f"⚠️ Erro ao verificar tamanho: {e}")

✅ Analytics gerais: 6 dias de dados

📊 RESUMO DOS DADOS RECOLHIDOS:
   💰 Vendas totais: 3897.92€
   📦 Encomendas totais: 22
   👥 Clientes totais: 11
   🛍️ Produtos ativos: 12
   💳 Encomendas pagas: 18
   🎯 Top produtos: 10
   📈 Meses com dados: 7

✅ Dados exportados para: ..\static\data\analytics.json
📄 Tamanho do ficheiro JSON: 13598 caracteres


# Export to JSON

Combine all data and save to analytics.json file.

In [7]:
# =============================================================================
# GENERAL ANALYTICS & EXPORT TO JSON
# =============================================================================

# Sales by month (last 12 months)
analytics_data["analytics"]["sales_by_month"] = analytics_data["orders"]["orders_by_month"]

# Orders by day (last 30 days)
query = """
SELECT 
    DATE(order_date) as order_day,
    COUNT(*) as order_count,
    SUM(total) as daily_revenue
FROM orders
WHERE order_date >= DATE_SUB(NOW(), INTERVAL 30 DAY)
GROUP BY DATE(order_date)
ORDER BY order_day
"""
result = safe_query(query)
for row in result:
    analytics_data["analytics"]["orders_by_day"].append({
        "date": row[0].isoformat() if row[0] else None,
        "count": row[1],
        "revenue": float(row[2]) if row[2] else 0
    })

# Top products (same as products section)
analytics_data["analytics"]["top_products"] = analytics_data["products"]["top_selling_products"]

# Customer analytics
analytics_data["analytics"]["customer_analytics"] = analytics_data["users"]["top_customers"]

# Payment methods
analytics_data["analytics"]["payment_methods"] = analytics_data["orders"]["orders_by_payment_method"]

# Monthly revenue (same as sales by month)
analytics_data["analytics"]["monthly_revenue"] = [
    {"month": item["month"], "revenue": item["revenue"]} 
    for item in analytics_data["orders"]["orders_by_month"]
]

print(f"✅ Analytics gerais: {len(analytics_data['analytics']['orders_by_day'])} dias de dados")

print(f"\n📊 RESUMO DOS DADOS RECOLHIDOS:")
print(f"   💰 Vendas totais: {analytics_data['dashboard']['total_sales']}€")
print(f"   📦 Encomendas totais: {analytics_data['dashboard']['total_orders']}")
print(f"   👥 Clientes totais: {analytics_data['dashboard']['total_customers']}")
print(f"   🛍️ Produtos ativos: {analytics_data['dashboard']['active_products']}")
print(f"   💳 Encomendas pagas: {analytics_data['orders']['paid_orders']}")
print(f"   🎯 Top produtos: {len(analytics_data['products']['top_selling_products'])}")
print(f"   📈 Meses com dados: {len(analytics_data['orders']['orders_by_month'])}")
print(f"   📦 Stock: {analytics_data['products']['out_of_stock_count']} esgotados, {analytics_data['products']['low_stock_count']} stock baixo")

# Export to JSON with proper decimal handling
output_path = os.path.join("..", "static", "data", "analytics.json")


def decimal_default(obj):
    """JSON serializer for objects not serializable by default json code"""
    if hasattr(obj, '__float__'):
        return float(obj)
    raise TypeError

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(analytics_data, f, indent=2, ensure_ascii=False, default=decimal_default)

print(f"\n✅ Dados exportados para: {output_path}")

# Check file size by reading it back
try:
    with open(output_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    print(f"📄 Tamanho do ficheiro JSON: {len(file_content)} caracteres")
except Exception as e:
    print(f"⚠️ Erro ao verificar tamanho: {e}")

print("\n🎉 Analytics com stock management atualizado!")
print("📊 Threshold de stock baixo: ≤ 10 unidades")
print("📦 Dados de stock incluídos no JSON para dashboard admin")

⚠️ Conexão perdida. A reconectar...
✅ Reconexão bem-sucedida
✅ Analytics gerais: 12 dias de dados

📊 RESUMO DOS DADOS RECOLHIDOS:
   💰 Vendas totais: 3897.92€
   📦 Encomendas totais: 22
   👥 Clientes totais: 11
   🛍️ Produtos ativos: 12
   💳 Encomendas pagas: 18
   🎯 Top produtos: 10
   📈 Meses com dados: 7
   📦 Stock: 0 esgotados, 0 stock baixo

✅ Dados exportados para: ..\static\data\analytics.json
📄 Tamanho do ficheiro JSON: 14145 caracteres

🎉 Analytics com stock management atualizado!
📊 Threshold de stock baixo: ≤ 10 unidades
📦 Dados de stock incluídos no JSON para dashboard admin


---

## 📈 **GENERAL ANALYTICS & EXPORT**

This section consolidates all analytics data and exports it to JSON for frontend consumption.

**Data Consolidation:**
- 📊 **Sales by Month** - Monthly revenue trends (12 months)
- 📅 **Orders by Day** - Daily order patterns (30 days)  
- 🔥 **Top Products** - Reference to best sellers
- 👑 **Customer Analytics** - VIP customer data
- 💳 **Payment Methods** - Payment preference summary
- 💰 **Monthly Revenue** - Revenue trend data

**Tables used:** products, categories, order_items

### **Export Process:**
1. **Data Validation** - Ensures all metrics are calculated
2. **JSON Serialization** - Converts to web-friendly format
3. **File Export** - Saves to `/static/data/analytics.json`

### **Frontend Integration:**
- Used by `analytics.js` for real-time dashboard updates
- Accessible via Flask route `/admin/refresh-analytics`
- Powers all admin page statistics and charts

In [8]:
# 🔒 Fechar conexão à base de dados
try:
    cursor.close()
    conn.close()
    print("✅ Conexão fechada com sucesso!")
except:
    print("⚠️ Conexão já estava fechada")

print("\n🎉 Processo concluído!")
print("💡 Agora podes usar os dados em JavaScript com:")
print("   fetch('/static/data/analytics.json')")

✅ Conexão fechada com sucesso!

🎉 Processo concluído!
💡 Agora podes usar os dados em JavaScript com:
   fetch('/static/data/analytics.json')


# Cleanup

Close database connection.